# 电子积分准备

同[非周期性教程](../qcbasic/basis_integral.ipynb)一样，我们先来准备一个尽可能小的体系。此处选择的是金刚石的最小重复单元，含两个碳原子。由于使用了GTH赝势，只有8个电子。

不过周期性的核排斥能量的计算没有非周期性那么简单，因此我们先来看电子积分。

In [1]:
from pyscf.pbc import gto

cell = gto.Cell()
cell.atom='''
C 0.000000000000   0.000000000000   0.000000000000
C 1.685068664391   1.685068664391   1.685068664391
'''
cell.a = '''
0.000000000, 3.370137329, 3.370137329
3.370137329, 0.000000000, 3.370137329
3.370137329, 3.370137329, 0.000000000'''
cell.unit = 'B'
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.build()

In [6]:
cell.nelec

(4, 4)

GTH-SZV 是一个足够小的基组，对于目前体系只有8个基函数。

In [7]:
cell._basis

{'C': [[0,
   [4.3362376436, 0.1490797872],
   [1.2881838513, -0.0292640031],
   [0.4037767149, -0.688204051],
   [0.1187877657, -0.3964426906]],
  [1,
   [4.3362376436, -0.0878123619],
   [1.2881838513, -0.27755603],
   [0.4037767149, -0.4712295093],
   [0.1187877657, -0.4058039291]]]}

In [8]:
cell.nao

8

## 重叠积分